In [10]:
!pip install --upgrade pandas
!pip install konlpy
!cp -R /home/work/deep/jdk1.8.0_201 /usr/lib/jvm
from IPython.display import clear_output
import pandas as pd
import numpy as np
import json
import os
from pprint import pprint
from konlpy.tag import Okt
clear_output()

## 원유가격 Load

In [11]:
df_oil_price = pd.read_csv('국제_원유가격20080607_20190610.csv', encoding='UTF-8')
df_oil_price.columns = ['time','dubai','brent','WIT']

df_oil_price['time'] = '20'+df_oil_price.time
df_oil_price['time'] = pd.to_datetime(df_oil_price['time'], format='%Y년%m월%d일')
df_oil_price.dubai = np.where(df_oil_price.dubai == '-', df_oil_price.dubai.shift(-1) , df_oil_price.dubai)
df_oil_price.dubai = np.where(df_oil_price.dubai == '-', df_oil_price.dubai.shift(-2) , df_oil_price.dubai)
df_oil_price.brent = np.where(df_oil_price.brent == '-', df_oil_price.brent.shift(-1) , df_oil_price.brent)
df_oil_price.brent = np.where(df_oil_price.brent == '-', df_oil_price.brent.shift(-2) , df_oil_price.brent)
df_oil_price.WIT = np.where(df_oil_price.WIT == '-', df_oil_price.WIT.shift(-1) , df_oil_price.WIT)
df_oil_price.WIT = np.where(df_oil_price.WIT == '-', df_oil_price.WIT.shift(-2) , df_oil_price.WIT)
df_oil_price.dubai = df_oil_price.dubai.astype(float)
df_oil_price.brent = df_oil_price.brent.astype(float)
df_oil_price.WIT = df_oil_price.WIT.astype(float)

### 1) 2017~2019 년도 뉴스데이터와 두바이 유 가격을 통해 상승/하락(긍정/부정)을 예측

In [12]:
df_oil_price = df_oil_price[df_oil_price.time >= '2017-01-01'][['time','dubai']]
df_oil_price.dubai - df_oil_price.dubai.shift(1)
#상승 / 하락을 체크하기 위해 하루 전 데이터와 차이를 구함
df_oil_price['trend'] = df_oil_price.dubai - df_oil_price.dubai.shift(1)
df_oil_price = df_oil_price.dropna(how='any')
#0보다 크면 상승(0), 0보다 작으면(1)하락
df_oil_price['trend_ca']= np.where(df_oil_price.trend > 0 , 0, 1)

### 2)  2017~2019 년도 뉴스데이터 Load

In [13]:
df_text = pd.read_csv('data/NewsMainOutput_dic.csv', encoding='UTF-8',sep='|')
df_text = df_text.dropna(how='any')
df_text['main']= df_text.main.str.replace("\t","")
df_text.time = np.where(df_text.time == '5일 전', '2019.06.09.' , df_text.time)
df_text.time = np.where(df_text.time == '6일 전', '2019.06.08.' , df_text.time)
df_text.time = np.where(df_text.time == '7일 전', '2019.06.07.' , df_text.time)
df_text.time = np.where(df_text.time == '4일 전', '2019.06.10.' , df_text.time)
df_text.time = np.where(df_text.time == '3일 전', '2019.06.11.' , df_text.time)
df_text.time = np.where(df_text.time == '2일 전', '2019.06.12.' , df_text.time)
df_text.time = pd.to_datetime(df_text['time'], format='%Y.%m.%d.')
df_text = pd.merge(df_text[['time','main']],df_oil_price,how='right',on='time').dropna(how='any')[['main','trend_ca']]

### 3) Test Train 으로 분할

In [14]:
train_data = df_text['main'][:1000]
train_y = df_text['trend_ca'][:1000]
test_data = df_text['main'][1000:]
test_y = df_text['trend_ca'][1000:]

In [15]:
okt = Okt()
def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

if os.path.isfile('train_docs.json'):
    with open('train_docs.json') as f:
        train_docs = json.load(f)
    with open('test_docs.json') as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
    test_docs = [(tokenize(row[1]), row[2]) for row in test_data]
    # JSON 파일로 저장
    with open('train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))
import nltk
text = nltk.Text(tokens, name='NMSC')

# 전체 토큰의 개수
print(len(text.tokens))

# 중복을 제외한 토큰의 개수
print(len(set(text.tokens)))            

# 출현 빈도가 높은 상위 토큰 10개
pprint(text.vocab().most_common(10))
selected_words = [f[0] for f in text.vocab().most_common(10000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')
x_train.shape

870
870
161
[('하다/Verb', 75),
 ('머/Noun', 57),
 ('이/Noun', 55),
 ('국/Noun', 47),
 ('제/Noun', 42),
 ('뉴/Noun', 36),
 ('서다/Verb', 30),
 ('앵/Adverb', 28),
 ('사/Noun', 25),
 ('우/Adverb', 22)]


(1000, 161)

### 4) 모델링/ 학습

In [27]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from keras import backend as K
from keras import layers
from keras import backend as K
from keras import optimizers
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.preprocessing import image

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from datetime import datetime

In [29]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(161,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=300, batch_size=512)

Epoch 1/300
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6828 - binary_accuracy: 0.6150
Epoch 2/300
1000/1000 [==============================] - 0s 8us/step - loss: 0.6706 - binary_accuracy: 0.6360
Epoch 3/300
1000/1000 [==============================] - 0s 8us/step - loss: 0.6628 - binary_accuracy: 0.6360
Epoch 4/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.6566 - binary_accuracy: 0.6360
Epoch 5/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.6518 - binary_accuracy: 0.6360
Epoch 6/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.6476 - binary_accuracy: 0.6360
Epoch 7/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.6441 - binary_accuracy: 0.6360
Epoch 8/300
1000/1000 [==============================] - 0s 5us/step - loss: 0.6413 - binary_accuracy: 0.6360
Epoch 9/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.6386 - binary_accuracy: 0.6360
Epoch 10/3

1000/1000 [==============================] - 0s 7us/step - loss: 0.5325 - binary_accuracy: 0.7020
Epoch 75/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.5321 - binary_accuracy: 0.7030
Epoch 76/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.5315 - binary_accuracy: 0.7030
Epoch 77/300
1000/1000 [==============================] - 0s 7us/step - loss: 0.5308 - binary_accuracy: 0.7030
Epoch 78/300
1000/1000 [==============================] - 0s 7us/step - loss: 0.5327 - binary_accuracy: 0.7040
Epoch 79/300
1000/1000 [==============================] - 0s 7us/step - loss: 0.5307 - binary_accuracy: 0.7020
Epoch 80/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.5309 - binary_accuracy: 0.7010
Epoch 81/300
1000/1000 [==============================] - 0s 7us/step - loss: 0.5295 - binary_accuracy: 0.7020
Epoch 82/300
1000/1000 [==============================] - 0s 7us/step - loss: 0.5303 - binary_accuracy: 0.6960
Epoch 83/300
1

1000/1000 [==============================] - 0s 7us/step - loss: 0.5259 - binary_accuracy: 0.7060
Epoch 148/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.5239 - binary_accuracy: 0.7000
Epoch 149/300
1000/1000 [==============================] - 0s 7us/step - loss: 0.5238 - binary_accuracy: 0.6990
Epoch 150/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.5230 - binary_accuracy: 0.6990
Epoch 151/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.5235 - binary_accuracy: 0.7030
Epoch 152/300
1000/1000 [==============================] - 0s 7us/step - loss: 0.5255 - binary_accuracy: 0.6970
Epoch 153/300
1000/1000 [==============================] - 0s 5us/step - loss: 0.5233 - binary_accuracy: 0.7000
Epoch 154/300
1000/1000 [==============================] - 0s 5us/step - loss: 0.5247 - binary_accuracy: 0.7020
Epoch 155/300
1000/1000 [==============================] - 0s 5us/step - loss: 0.5233 - binary_accuracy: 0.7000
Epoch 

1000/1000 [==============================] - 0s 7us/step - loss: 0.5224 - binary_accuracy: 0.6970
Epoch 221/300
1000/1000 [==============================] - 0s 5us/step - loss: 0.5216 - binary_accuracy: 0.6990
Epoch 222/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.5225 - binary_accuracy: 0.7030
Epoch 223/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.5227 - binary_accuracy: 0.7010
Epoch 224/300
1000/1000 [==============================] - 0s 7us/step - loss: 0.5246 - binary_accuracy: 0.6960
Epoch 225/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.5242 - binary_accuracy: 0.7040
Epoch 226/300
1000/1000 [==============================] - 0s 5us/step - loss: 0.5229 - binary_accuracy: 0.7040
Epoch 227/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.5219 - binary_accuracy: 0.7050
Epoch 228/300
1000/1000 [==============================] - 0s 5us/step - loss: 0.5220 - binary_accuracy: 0.6970
Epoch 

1000/1000 [==============================] - 0s 5us/step - loss: 0.5259 - binary_accuracy: 0.7040
Epoch 294/300
1000/1000 [==============================] - 0s 5us/step - loss: 0.5222 - binary_accuracy: 0.6970
Epoch 295/300
1000/1000 [==============================] - 0s 5us/step - loss: 0.5251 - binary_accuracy: 0.7040
Epoch 296/300
1000/1000 [==============================] - 0s 5us/step - loss: 0.5223 - binary_accuracy: 0.6970
Epoch 297/300
1000/1000 [==============================] - 0s 5us/step - loss: 0.5223 - binary_accuracy: 0.7020
Epoch 298/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.5235 - binary_accuracy: 0.6990
Epoch 299/300
1000/1000 [==============================] - 0s 6us/step - loss: 0.5218 - binary_accuracy: 0.7020
Epoch 300/300
1000/1000 [==============================] - 0s 5us/step - loss: 0.5223 - binary_accuracy: 0.7010


In [30]:
results = model.evaluate(x_test, y_test)
results

319/319 [==============================] - 0s 220us/step


[1.1392261922172617, 0.8181818187423634]

In [32]:
df = pd.read_csv('NewsMainOutput.csv', encoding='UTF-8',sep='|')
df['main']= df.main.str.replace("\t","")
df['main']= df['main'].str.replace('[','')
df['main']= df['main'].str.replace(']','')
df['main']= df['main'].str.replace('서울신문','')
df.dropna(how='any',inplace=True) 
#df['main']= df['main'].astype('str')
data = {'time': df.time.unique(),'main' : df.groupby(['time'])['main'].apply(lambda x: "%s" % ', '.join(x)).values}
df = pd.DataFrame(data=data)

In [33]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    positive.append(score)
    negative.append(1 - score)
positive = []
negative = []
for i in range(1321):
    predict_pos_neg(df['main'][i])
df['positive'] = positive
df['negative'] = negative
df = df[['time','positive','negative']]

In [37]:
df['time'] = pd.to_datetime(df.time, format='%Y.%m.%d.')

In [38]:
df.to_csv('news_pn.csv',encoding='UTF-8')